In [1]:
import json

with open('tr_token_cosmos_mapping.json') as f:
    token_list = json.load(f)

In [2]:
len(token_list)

30158

In [3]:
from transformers import GPT2LMHeadModel

cosmos_model = GPT2LMHeadModel.from_pretrained("ytu-ce-cosmos/turkish-gpt2")
cosmos_model

/Users/alibayram/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


config.json:   0%|          | 0.00/893 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
import torch

from transformers import Gemma2Model, Gemma2ForCausalLM

gemma_model = Gemma2Model.from_pretrained("google/gemma-2-2b-it")
gemma_model

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Gemma2Model(
  (embed_tokens): Embedding(256000, 2304, padding_idx=0)
  (layers): ModuleList(
    (0-25): 26 x Gemma2DecoderLayer(
      (self_attn): Gemma2Attention(
        (q_proj): Linear(in_features=2304, out_features=2048, bias=False)
        (k_proj): Linear(in_features=2304, out_features=1024, bias=False)
        (v_proj): Linear(in_features=2304, out_features=1024, bias=False)
        (o_proj): Linear(in_features=2048, out_features=2304, bias=False)
        (rotary_emb): Gemma2RotaryEmbedding()
      )
      (mlp): Gemma2MLP(
        (gate_proj): Linear(in_features=2304, out_features=9216, bias=False)
        (up_proj): Linear(in_features=2304, out_features=9216, bias=False)
        (down_proj): Linear(in_features=9216, out_features=2304, bias=False)
        (act_fn): PytorchGELUTanh()
      )
      (input_layernorm): Gemma2RMSNorm((2304,), eps=1e-06)
      (pre_feedforward_layernorm): Gemma2RMSNorm((2304,), eps=1e-06)
      (post_feedforward_layernorm): Gemma2RMSNorm((2304,),

In [6]:
import torch
# create an empty tensor to store the embeddings of the tokens shape (len(token_list), gemma_model.embed_tokens.weight.shape[1])
cosmos_embeddings = cosmos_model.transformer.wte.weight
embeddings = torch.zeros(len(token_list), cosmos_embeddings.shape[1])
embeddings.shape

torch.Size([30158, 768])

In [ ]:
# create an empty tensor to store the embeddings of the tokens shape (len(token_list), gemma_model.embed_tokens.weight.shape[1])
gemma_embeddings = gemma_model.embed_tokens.weight
embeddings = torch.zeros(len(token_list), gemma_embeddings.shape[1])
embeddings.shape

torch.Size([30158, 2304])

In [7]:
token_list[0], cosmos_embeddings[8714]

({'tr_token': 'Sperm', 'tr_token_id': 25670, 'cosmos_token_ids': [51, 23314]},
 tensor([ 4.0595e-02,  1.8800e-02, -5.6896e-02, -4.4745e-03, -2.0354e-02,
         -2.9188e-02,  2.6052e-02, -3.0005e-02,  6.4966e-03, -4.7019e-02,
          1.4762e-02,  4.5944e-02,  6.9625e-03, -1.7376e-02, -4.6436e-02,
          3.1336e-02, -4.8943e-02, -2.1498e-02, -3.5814e-02,  1.9699e-02,
          2.5365e-02, -4.2383e-02, -5.6480e-02, -3.6583e-02,  1.9868e-02,
          1.7135e-02, -2.0939e-02, -2.2281e-02,  4.0893e-02, -3.9063e-02,
          2.9882e-03, -1.8646e-02,  6.0204e-02, -2.9248e-02,  5.9868e-02,
         -7.0782e-03,  4.5196e-02,  3.4762e-02,  3.2854e-02,  1.9579e-02,
         -2.5731e-02,  1.2262e-02,  2.3121e-02,  3.1659e-03,  6.1822e-02,
         -1.5077e-02, -1.5829e-02, -2.1639e-02,  4.4487e-02, -1.9498e-02,
          9.1932e-03,  2.4858e-02,  1.5707e-02,  4.5323e-03, -2.0508e-02,
         -4.0508e-02,  2.3951e-02,  1.6167e-02,  9.0063e-03,  5.9529e-02,
          2.6293e-02,  4.1352e-03

In [8]:
cosmos_model.can_generate()

True

In [27]:
e1 = gemma_embeddings[7871]
e2 = gemma_embeddings[58714]

e1 = e1 + e2
average = e1 / 2
average

tensor([-0.0067, -0.0076,  0.0065,  ...,  0.0170, -0.0135,  0.0192],
       grad_fn=<DivBackward0>)

In [11]:
# for each token in the token_list, get the corresponding embedding from the gemma model and store it in the embeddings tensor
# if there is more than one token in the token_list that maps to the same index, average the embeddings

for token_map in token_list:
    index = token_map['tr_token_id']
    cosmos_token_ids = token_map['cosmos_token_ids']
    embedding = cosmos_embeddings[cosmos_token_ids[0]]
    sum_embedding = embedding
    for cosmos_token_id in cosmos_token_ids[1:]:
        embedding = embedding + cosmos_embeddings[cosmos_token_id]
    if len(cosmos_token_ids) > 1:
        embedding = embedding / len(cosmos_token_ids)        
    embeddings[index] = embedding

embeddings[0]

tensor([ 3.1843e-02, -5.5590e-03,  2.3134e-02,  2.5998e-02,  3.8133e-02,
        -1.1781e-02, -4.4985e-02,  7.0144e-03,  8.3220e-03, -1.2933e-02,
        -3.0127e-03,  3.9123e-02, -5.4939e-03, -6.3854e-04,  3.7306e-02,
        -6.6267e-03,  4.3927e-03,  4.6355e-03,  6.5963e-04, -3.6407e-02,
        -1.2556e-03,  3.9117e-02,  1.6696e-03, -1.7141e-02, -3.9747e-03,
        -7.4159e-03, -1.0277e-02, -9.0470e-03, -7.3650e-04,  3.1024e-02,
         2.6026e-02, -3.7019e-02,  2.6812e-02,  1.1434e-02, -2.3057e-02,
        -7.8760e-03,  2.6135e-02,  1.0838e-02,  2.1567e-02,  5.7776e-03,
        -3.7025e-03,  4.3307e-02, -1.3431e-02,  6.3660e-03,  1.0688e-02,
        -1.3676e-02, -9.5783e-03,  1.6308e-02, -1.2501e-03, -9.1958e-03,
        -1.4645e-02,  2.5962e-02, -1.6468e-03, -1.7224e-02, -2.5646e-02,
        -2.8907e-02,  1.0940e-02,  1.7490e-02,  4.9958e-02, -1.1820e-02,
        -3.1167e-02,  1.3588e-02,  3.5271e-03, -2.0744e-02,  2.7477e-02,
         1.0007e-02,  1.1308e-02, -3.6941e-02, -5.2

In [12]:
count_of_zero_embeddings = 0
for i in range(len(embeddings)):
    if torch.all(embeddings[i] == 0):
        count_of_zero_embeddings += 1
count_of_zero_embeddings

0

In [13]:
# save the embeddings tensor to a file
torch.save(embeddings, 'tr_cosmos_embeddings.pt')

In [40]:
# change gemma2_model.embed_tokens.weight to the embeddings tensor
gemma_model.embed_tokens.weight = torch.nn.Parameter(embeddings)


In [41]:
gemma_model.config.vocab_size = len(token_list)

In [42]:
gemma_model.save_pretrained("tr_gemma2_model")

In [43]:
gemma2_model = Gemma2ForCausalLM.from_pretrained("tr_gemma2_model")
gemma2_model

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Gemma2ForCausalLM(
  (model): Gemma2Model(
    (embed_tokens): Embedding(30158, 2304, padding_idx=0)
    (layers): ModuleList(
      (0-25): 26 x Gemma2DecoderLayer(
        (self_attn): Gemma2Attention(
          (q_proj): Linear(in_features=2304, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2304, out_features=1024, bias=False)
          (v_proj): Linear(in_features=2304, out_features=1024, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2304, bias=False)
          (rotary_emb): Gemma2RotaryEmbedding()
        )
        (mlp): Gemma2MLP(
          (gate_proj): Linear(in_features=2304, out_features=9216, bias=False)
          (up_proj): Linear(in_features=2304, out_features=9216, bias=False)
          (down_proj): Linear(in_features=9216, out_features=2304, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): Gemma2RMSNorm((2304,), eps=1e-06)
        (pre_feedforward_layernorm): Gemma2RMSNorm((2304,), eps=

In [86]:
from transformers import AutoTokenizer

tr_tokenizer = AutoTokenizer.from_pretrained("alibayram/tr_tokenizer")
tr_tokenizer.is_fast

True

In [88]:
tr_tokenizer.vocab

{'Vazife': 23130,
 'Soydaş': 10179,
 'Idea': 21996,
 'isinden': 4822,
 'Sön': 16120,
 'ilçesi': 6547,
 'ing': 1502,
 'Demirle': 9012,
 'çam': 25674,
 'Abadi': 16870,
 'Eurov': 3582,
 'imren': 9547,
 'lanması': 4245,
 'amorf': 7701,
 'Mazlum': 21406,
 'Ayva': 22072,
 'Ε': 317,
 'üslup': 12884,
 'Traş': 26903,
 'Orhan': 6648,
 'abone': 25438,
 'Manuel': 25481,
 'Tavsiye': 12814,
 'galibiyet': 6794,
 'asistan': 18870,
 'kalker': 10059,
 'Kamu': 8032,
 'önemli': 2040,
 'mahmuz': 26575,
 'gevşek': 28937,
 'Aman': 22982,
 'Bezelye': 14278,
 'Doğaçlama': 29802,
 'ҡ': 446,
 'Levent': 13232,
 'esma': 19136,
 'Çapar': 22434,
 'Kaçamak': 22240,
 'hısım': 27112,
 'Pey': 13201,
 'eradikasyon': 26666,
 'tures': 6898,
 'Kayak': 15490,
 'Şekillen': 11255,
 'Kümülüs': 27612,
 'Zıt': 27371,
 'kazağı': 25353,
 'iliş': 6099,
 'sın': 6055,
 'Elvan': 30011,
 'düş': 1731,
 'Ortala': 19641,
 'başlangıç': 16210,
 'tab': 2380,
 'Marş': 12173,
 'kısrak': 25990,
 'Bedeni': 19351,
 'bekleme': 12946,
 '부': 1142,
 '

In [69]:
# Load model directly
from transformers import GemmaTokenizerFast

gemma2_tokenizer = GemmaTokenizerFast.from_pretrained("google/gemma-2-2b-it")
gemma2_tokenizer.is_fast

True

In [75]:
prompt = "aa"

In [78]:
inputs = gemma2_tokenizer(prompt, return_tensors="pt")

In [52]:
gemma2_model.generation_config

GenerationConfig {
  "bos_token_id": 2,
  "cache_implementation": "hybrid",
  "eos_token_id": [
    1,
    107
  ],
  "pad_token_id": 0
}

In [79]:
inputs

{'input_ids': tensor([[   2, 6499]]), 'attention_mask': tensor([[1, 1]])}

In [80]:
outputs = gemma2_model.generate(**inputs)


Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


In [83]:
gemma2_tokenizer.decode(outputs[0])

'<bos>aa<2mass><2mass><eos>'

In [92]:
import torch

device = torch.device("mps")
device

device(type='mps')

In [90]:
device.type

'mps'

In [97]:
%pip install torch torchvision torchaudio



huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [96]:
%python -m pip install --upgrade pip

UsageError: Line magic function `%python` not found (But cell magic `%%python` exists, did you mean that instead?).
